In [1]:
import pandas as pd

#read csv files
df_f=pd.read_csv("follows.csv")
df_i=pd.read_csv("interests.csv")

df_f.columns = df_f.columns.str.strip()
df_i.columns = df_i.columns.str.strip()

#one-hot coding
df_i = pd.get_dummies(df_i,columns=['category'])
df_i = df_i.groupby('user_id').sum()

df_f = pd.get_dummies(df_f,columns=['followee_id'])
df_f = df_f.groupby('follower_id').sum()
df_f.index.names = ['user_id']

In [2]:
#rename df_f columns
df_f_cols= [df_f.columns.tolist()[i].strip('followee_id_') for i in range(len(df_f.columns.tolist()))]
df_f.columns= df_f_cols

In [3]:
#inner join two make both dataframes have same user_ids
df_test1 = pd.DataFrame.join(df_i,df_f,on='user_id',how='inner')
df_test1.shape

(2237, 4478)

In [4]:
#transpose datasets
df_reduced_i = df_test1.T.iloc[:883].T
df_reduced_f = df_test1.T.iloc[883:].T

In [5]:
#transpose datasets
df_f_t = df_reduced_f.T
df_f_t.index.names=['followee_id']
df_f_t.index=df_f_t.index.astype('int64')
df_reduced_i.index.names = ['followee_id']

In [6]:
#make followees are same as user_ids
df_test2 = pd.DataFrame.merge(df_reduced_i,df_f_t,left_index=True, right_index=True)
df_test2.index.names=['user_id']
df_test2.shape

(2237, 3120)

In [7]:
#transpose back to origin dataframes
df_interest = df_test2.T.iloc[:883].T
df_followee = df_test2.T.iloc[883:].T

In [8]:
import numpy as np

#create matrix and calculate correlation
imatrix = df_interest.values
icorr = np.corrcoef(imatrix)
icorr.shape

(2237, 2237)

In [9]:
user_id = df_interest.index.values
followee_id = df_followee.columns.values

#create predicition matrix for recommended followees
df_recom_uf = pd.DataFrame(columns =df_followee.columns.tolist(), index =df_followee.index.tolist())

In [10]:
#Correlation model starts
for n in range(0, icorr.shape[0]):
    
    #create a df to sort corr of each user to user n
    corrdata = {'user_id': user_id, 'corr': icorr[n]}
    df_corr = pd.DataFrame(data=corrdata)
    df_corr = df_corr.sort_values(by=['corr'], axis=0, ascending=False)
    
    #get 5 users with highest corr score which means they are the most similar user to user n
    recom_corr = df_corr[:5]
    #calculate weights of each 5 users
    recom_corr['sum'] = [recom_corr['corr'].sum()] * 5
    recom_corr['proportion'] = recom_corr['corr'] / recom_corr['sum']
    #get followees of 5 users
    df_user_filtering = pd.DataFrame.join(recom_corr,df_followee,on='user_id',how='inner')
    
    #calculate recommendation scores of each user (2237) by multiplying proportion column and value of each user
    sum_uf = df_user_filtering.iloc[:, 2]
    score_uf = []
    for m in range(4, 2241):
        interest_uf = df_user_filtering.iloc[:, m]
        total_uf = sum((sum_uf * interest_uf).tolist())
        score_uf.append(total_uf)
    
    #create a df to sort recommendation score of 2237 users
    re_uf = {'user_id': followee_id, 'score': score_uf}
    df_uf = pd.DataFrame(data=re_uf)
    df_uf = df_uf.sort_values(by=['score'], axis=0, ascending=False)
    
    #get number of followees n already followed
    nrecommendation = df_followee.loc[followee_id[n]].sum()
    #get top nth recommenders
    recom_uf = df_uf[:nrecommendation]
    recom_uf_id = recom_uf.user_id.values.tolist()
    
    #insert value into recommendation matrix
    df_recom_uf.at[followee_id[n], recom_uf_id] = 1
    #df_recom_uf.iloc[n].value_counts()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [11]:
from sklearn.metrics import precision_score
from statistics import mean
def prec_score(df_true, df_prediction): 
    pre = [precision_score(df_true.loc[i],df_prediction.loc[i])for i in df_prediction.index.values ]
    return mean(pre)

In [12]:
from sklearn.metrics import f1_score
def fone_score(df_true, df_prediction):
    f1 = [f1_score(df_true.loc[i],df_prediction.loc[i]) for i in df_prediction.index.values]
    return mean(f1)

In [15]:
#prediction/ recommended followee
df_pred = df_recom_uf.fillna(0)
df_true = df_followee

In [16]:
print("precision_score is: ",prec_score(df_true,df_pred))
print("f1_score is: ",fone_score(df_true,df_pred))

precision_score is:  0.856619686079582
f1_score is:  0.856619686079582
